In [8]:
import numpy as np
from json import load
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, recall_score
import joblib
nlp = spacy.load("ru_core_news_sm")


In [9]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [10]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [11]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


In [13]:
gaussian_naive_bayes_optimal = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),},
                                                        scoring=recall_score).fit(X_train, y_train)
gaussian_naive_bayes_optimal.best_params_
y_predicted_by_gaussian_naive_bayes = gaussian_naive_bayes_optimal.predict(X_test)


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: recall_score() takes 2 positional arguments but 3 were given

  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: recall_score() 

In [14]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes))



Accuracy
0.8913857677902621
Confusion matrix
[[ 85  11]
 [ 18 153]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.83      0.89      0.85        96
           1       0.93      0.89      0.91       171

    accuracy                           0.89       267
   macro avg       0.88      0.89      0.88       267
weighted avg       0.89      0.89      0.89       267

